In [44]:
import pandas as pd
messages = pd.read_csv("SMSSpamCollection", delimiter='\t', names=["label", "message"])
messages

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [45]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stopwords = stopwords.words("english")
stemming = PorterStemmer()

In [46]:
corpus = []
for i in range(0, len(messages)):
    review = re.sub(r"[^ a-z A-Z]", " ", messages["message"][i])
    review = re.sub(r"/s+"," ", review)
    review = review.lower()
    review = review.split()
    review = [stemming.stem(word) for word in review if word not in stopwords]
    review = " ".join(review)
    corpus.append(review)

In [54]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000)
X = cv.fit_transform(corpus).toarray()
y = pd.get_dummies(messages['label'])
print(y.value_counts())
y = y.iloc[ : ,1].values

print(X.shape)
print(y.shape)

ham  spam
1    0       4825
0    1        747
dtype: int64
(5572, 5000)
(5572,)


In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(X_train.shape)
print(X_test.shape)

(4457, 5000)
(1115, 5000)


In [49]:
from sklearn.naive_bayes import MultinomialNB
spam_detection_model = MultinomialNB().fit(X_train, y_train)

y_pred = spam_detection_model.predict(X_test)

In [52]:
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)

[[946   9]
 [  8 152]]


In [53]:
from sklearn.metrics import accuracy_score
accuracy_score = accuracy_score(y_test, y_pred)
print(accuracy_score)

0.9847533632286996
